In [32]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-02-16 14:58:26,637: Credentials are already in use. The existing account in the session will be replaced.


In [33]:

provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [36]:
backend = provider.get_backend('ibmq_16_melbourne')
backend

<IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>

In [37]:
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import CobylaOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
qaoa = MinimumEigenOptimizer(QAOA(quantum_instance=backend))

In [39]:
from docplex.mp.model import Model
nbKids=150
costBus40=500
costBus30=400

model = Model()
#nbBus40b2 = model.binary_var(name='nbBus40b2')
nbBus40b1 = model.binary_var(name='nbBus40b1')
nbBus40b0 = model.binary_var(name='nbBus40b0')
#nbBus30b2 = model.binary_var(name='nbBus30b2')
nbBus30b1 = model.binary_var(name='nbBus30b1')
nbBus30b0 = model.binary_var(name='nbBus30b0')

model.minimize(#nbBus40b2*4*costBus40+
               nbBus40b1*2*costBus40+nbBus40b0*costBus40+
               #nbBus30b2*4*costBus30+
               nbBus30b1*2*costBus30+nbBus30b0*costBus30)
model.add_constraint(#nbBus40b2*4*40+
                     nbBus40b1*2*40+nbBus40b0*40+
                     #nbBus30b2*4*30+
                     nbBus30b1*2*30+nbBus30b0*30>=nbKids
                     , "nbKids")





# load quadratic program from docplex model
qp = QuadraticProgram()
qp.from_docplex(model)
print(qp.export_as_lp_string())

# run QAOA
result = qaoa.solve(qp)

bresult=result.x

print("x={}".format(bresult))
print("fval={:.2f}".format(result.fval))

print()

print("cost = ",result.fval)
#solnbbus40=4*bresult[0]+2*bresult[1]+bresult[2]
#solnbbus30=4*bresult[3]+2*bresult[4]+bresult[5]
solnbbus40=2*bresult[0]+bresult[1]
solnbbus30=2*bresult[2]+bresult[3]

print(solnbbus40, " buses 40 seats and ",solnbbus30, "buses 30 seats")

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model14

Minimize
 obj: 1000 nbBus40b1 + 500 nbBus40b0 + 800 nbBus30b1 + 400 nbBus30b0
Subject To
 nbKids: 80 nbBus40b1 + 40 nbBus40b0 + 60 nbBus30b1 + 30 nbBus30b0 >= 150

Bounds
 0 <= nbBus40b1 <= 1
 0 <= nbBus40b0 <= 1
 0 <= nbBus30b1 <= 1
 0 <= nbBus30b0 <= 1

Binaries
 nbBus40b1 nbBus40b0 nbBus30b1 nbBus30b0
End

x=[1. 1. 0. 1.]
fval=1900.00

cost =  1900.0
3.0  buses 40 seats and  1.0 buses 30 seats
